In [1]:
import numpy as np
import rasterio
from osgeo import gdal
from pysheds.grid import Grid

In [12]:
grid = Grid.from_raster('../IMGS/area_estudo/Teste_IV_dsm/DSM.tif', data_name='DSM')
grid.read_raster('../IMGS/area_estudo/Teste_IV_dsm/DSM.tif', data_name='dir')
grid.view('dir')

Raster([[685.42633, 685.4319 , 685.4372 , ..., 684.6238 , 684.6233 ,
         684.6226 ],
        [685.42755, 685.4326 , 685.43805, ..., 684.62366, 684.6233 ,
         684.62274],
        [685.4277 , 685.43317, 685.43805, ..., 684.62366, 684.62366,
         684.6235 ],
        ...,
        [684.7793 , 684.7783 , 684.76917, ..., 683.76495, 683.764  ,
         683.7628 ],
        [684.76666, 684.7634 , 684.761  , ..., 683.7644 , 683.76385,
         683.7628 ],
        [684.7576 , 684.75507, 684.7488 , ..., 683.7651 , 683.764  ,
         683.76337]], dtype=float32)

In [14]:
# Determine D8 flow directions from DEM
# ----------------------
# Fill depressions in DEM
grid.fill_depressions('DSM', out_name='flooded_dem')
    
# Resolve flats in DEM
grid.resolve_flats('flooded_dem', out_name='inflated_dem')
    
# Specify directional mapping
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    
# Compute flow directions
# -------------------------------------
grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
grid.view('dir')

Raster([[  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,  16.,  16., ...,   1.,   1.,   0.],
        [  0.,  16.,  16., ..., 128., 128.,   0.],
        ...,
        [  0.,   4.,   2., ...,   1.,   1.,   0.],
        [  0.,   2.,   4., ...,   1.,   1.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32)

In [ ]:
dataset_dem = gdal.Open('../IMGS/area_estudo/Teste_IV_dsm/DSM.tif')

In [ ]:
band_dem = dataset_dem.GetRasterBand(1)

In [ ]:
array_dem = band_dem.ReadAsArray()
array_dem

In [ ]:
array_dem_norm = (array_dem - np.min(array_dem)) / (np.max(array_dem) - np.min(array_dem))
array_dem_norm

In [ ]:
# get difference in x- and y- direction
sec_grad_x = np.diff(array_dem_norm,n=1,axis=0)
sec_grad_y = np.diff(array_dem_norm,n=1,axis=1)

cp = []
# starts from 1 because diff function gives a forward difference
for i in range(1,n-1):
    for j in range(1,n-1):
        # check when the difference changes its sign
        if ((sec_grad_x[i-1,j]<0) != (sec_grad_x[i-1+1,j]<0)) and \
           ((sec_grad_y[i,j-1]<0) != (sec_grad_y[i,j-1+1]<0)):
            cp.append([i,j,  storm[i,j]])

cp = np.array(cp)